# Requerimientos

In [106]:
import sys
!{sys.executable} -m pip install pycryptodome

# Ejemplo de una blockchain con Python 
En este ejemplo se va a desarrollar una cadena de bloques que haga lo siguiente:
- Trabaje con el consenso Proof of Work y Proof of Stake
- Incluya transacciones entre dos usuarios en la red.
- Incluya cifrado asimetrico de un solo camino

Lo primero es subdivir como lo vamos a desarrollar. 
(Muy problablemente)
Desarrollar una clase:
1. Transaction
2. Account
3. Block
4. Blockchain

Primero desarrollamos los elementos que interactuan dentro de la blockchain, para despues integrarlo todo en una clase Blockchain.

## Clase Block

Una bloque es donde se guarda la informacion en blockchain y se cifra. Los atributos principales, o que tienen mas relevancia serian los siguientes:

Numero del bloque: Altura del bloque actual en la cadena.
Hash del bloque anterior: Firma digital del bloque anterior en la cadena.
Lista de transacciones: Transacciones contenidas y procesadas.
Hash del bloque actual: Firma digital del bloque actual.

Aunque tambien tiene otros atributos:
- Merkle Root: Hash del estado actual de la blockchain.
- Timestamp: Hora en la que el bloque se añadio a la cadena de bloques.
- Nonce: Numero magico que resuelve el "acertijo" en el consenso Proof of Work.

En nuestro directorio de trabajo vamos a crear un archivo llamado block.py. Vamos a añadirle los atributos que necesitamos, y algunos metodos que nos van a ayudar en un futuro con algunas funciones de la blockchain.

In [107]:
from datetime import datetime, date
from account import Account


class Block:
    def __init__(self, previous_hash: str, list_of_transactions: list, block_number: int) -> None:

        # Atributos de la clase Block
        self.block_number = block_number 
        self.previous_hash = previous_hash 
        self.list_of_transactions = list_of_transactions 
        self.hash = 0 
        self.merkle_root = 0 
        self.nonce = 0 
        
        # Bucle for que recorre la list de transacciones y les asigna su numero de bloque
        for tx in self.list_of_transactions:
            tx.block = block_number

        # Hora en la que el bloque es creado
        now = datetime.now()
        time = now.strftime("%H:%M:%S")
        today = str(date.today())
        self.time_stamp = time + " " + today

ModuleNotFoundError: No module named 'account'

In [ ]:
from datetime import datetime, date
from account import Account


class Block:
    def __init__(self, previous_hash: str, list_of_transactions: list, block_number: int):
        # Atributos de la clase Block
        self.block_number = block_number
        self.previous_hash = previous_hash  
        self.list_of_transactions = list_of_transactions
        self.nonce = 0
        self.hash = 0
        self.merkle_root = 0
        
         # timestamp del bloque
        now = datetime.now()
        time = now.strftime("%H:%M:%S")
        today = str(date.today())
        self.time_stamp = time + " " + today

        # Bucle for que recorre la list de transacciones y les asigna su numero de bloque
        for tx in self.list_of_transactions:
            tx.block = block_number
       

    def get_block_header(self) -> dict:
        """" Funcion que retorna un diccionario con el encabezado de nuestro bloque.
        """
        return {
            'previous_block_hash':self.previous_hash,
            'nonce': self.nonce,
            'transactions':self.get_tx_in_format()
        }

    def print_block_info(self) -> None:
        """Funcion que imprime la informacion del bloque, pero bonito."""
        print("-------------")
        print("Bloque No: ", self.block_number)
        print("Transacciones: ")
        self.print_tx_in_format()
        print("Hash anterior: ", self.previous_hash)
        print("Hash actual: ", self.hash)
        print("Time stamp: ", self.time_stamp)

    def print_tx_in_format(self) -> None:
        """Funcion que imprime de manera secuencial, y en formato,
        las transacciones del bloque."""
        for tx in self.list_of_transactions:
            print(
                f"- {tx.sender.nickname} send {tx.value} to {tx.recipient.nickname}")
    
    def get_tx_in_format(self) -> str:
        """Funcion que regresa una cadena de caracteres, de una lista que contiene las transacciones
        del bloque."""
        tx_list = []
        for tx in self.list_of_transactions:
            tx_in_str = f"{tx.sender.nickname} send {tx.value} to {tx.recipient.nickname}"
            tx_list.append(tx_in_str)
        return str(tx_list)



## Clase Account

Una cuenta es el medio por el que un usuario puede interactuar en la blockchain, tener activos y hacer transacciones hacia otros usuarios. Pueden tener un nickname, tienen un balance y un historial que registra las transacciones que han realizado. En blockchain, las cuentas cuentan con cifrado asimetrico, en otras palabras, con llaves publicas y privadas.

Cada objeto Account tendra una llave publica que funge como un identificador publico para la cuenta, y una llave privada para autorizar transacciones de la cuenta. 

Vamos a añadir un archivo al directorio de trabajo llamado account.py. En el vamos a crear un objeto que nos ayuda a fungir el papel de cuenta en una red blockchain.

In [ ]:
import binascii
from Crypto.PublicKey import RSA
from Crypto.Signature.pkcs1_15 import PKCS115_SigScheme

class Account:
    def __init__(self, nickname: str):
        self.nickname = nickname
        self.balance = 100
        self.list_of_all_transactions = []
        # Cifrado asimetrico
        self.private_key = RSA.generate(1024) # Llave privada con algoritmo RSA de 1024 bites 
        self.public_key = self.private_key.publickey() # Llave publica 
        self.signer = PKCS115_SigScheme(self.private_key) # (1)
        self.verifier = PKCS115_SigScheme(self.public_key) # (2)

    @property
    def identity(self):
        return binascii.hexlify(self.public_key.exportKey(format="DER")).decode('ascii') # (3)


A nuestra clase Account le añadimos los atributos nickname, balance y su lista de transacciones. Pero vamos a prestarle mas atencion a los otros conceptos debajo del comentario **# Cifrado asimetrico.**

### Visualizacion de las llaves 
Primero observaremos las llaves que se instancian por cuenta. Con la bibloteca Crypto podemos usar el modulo PublicKey para tener acceso al algoritmo RSA. 

El como se visualizan las llaves, lo veran a continuacion.

In [ ]:
from Crypto.PublicKey import RSA
private_key = RSA.generate(1024)
private_key.exportKey() # LLave privada

In [ ]:
private_key.publickey().exportKey() # LLave publica

La llave privada se suele visualizar de otra manera, esta manera la vamos a llamar 'identidad'. 

### Identidad

Las llaves publicas son el identificador de una cuenta, pero no se presentan o se visualizan como se ve arriba. Hay que convertirlo a hexadecimal para que sea visiblemente mas accesible.

In [ ]:
import binascii
binascii.hexlify(private_key.publickey().exportKey(format="DER")).decode('ascii') # (3)
# este funcion convierte lo que exporta RSA en hexadecimal, haciendolo mas visible.

### Signer y Verifier de una cuenta

Cada cuenta va a tener, digamos, dos items. Una va a ser una pluma, o una "firmadora", que nos ayudara a "firmar" las transacciones. Y la otra es una "verificadora", que se asegura que esa firma es legitima y que ni el contenido, ni la firma, ni el remitente, hayan cambiado.

Derivadas de estas llaves, se obtienen estos items. (1 y 2)
- Para crear el signer, se necesita la llave privada de la cuenta para autorizan/firman las transacciones. 
- Para crear el verifier, se necesita la llave publica de la cuenta para verificar el contenido, autor y firma digital de la transaccion.

Con la bibloteca Crypto podemos usar el modulo Signature.pkcs1_15 para obtener un objeto de tipo PKCS115_SigScheme. Con el podemos instanciar nuestra firmadora y nuestra verificadora.

In [ ]:
firmadora = PKCS115_SigScheme(private_key)  # Se instancia con la llave privada
verificadora = PKCS115_SigScheme(private_key.publickey())  # Se instancia con la llave publica

En nuestra clase Account, cada cuenta va a contar con su propio firmador y verificador, mas adelante veremos como se aplican.

## Clase Transacccion

Una transaccion en el contexto de blockchain es un cambio de estado en la red. Las transacciones de nuestra blockchain permiten que dos usuarios puedan transferirse dinero virtual.

Cada transaccion tiene 3 elementos principales.
- Quien manda la transaccion (sender)
- Quien la recibe (recipient)
- Cuanto dinero es (value)

Y elementos "informativos", sirven para guardar informacion importante de la transaccion en si misma.

- Timestamp: Hora en la que la transaccion fue añadida a la red.
- Block: Bloque al que pertenece la transaccion.
- Signature: Firma digital de la transaccion. 
- Status: Estado actual de la transaccion.

Vamos a crear en nuestro directorio de trabajo un archivo llamado transaction.py. En el escribiremos los atributos y metodos que necesitamos.

In [ ]:
from Account import Account 

class Transaction:
    def __init__(self, sender: Account, value: int, recipient: Account):
        self.sender = sender
        self.value = value
        self.recipient = recipient
        self.time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        self.block = None
        self.signature = None
        self.status = TxStatus.PENDIENTE # (1)
        # Al instanciarse una tx, esta debe reflejarse en la cuenta que la envia. (Sender)
        sender.list_of_all_transactions.append(self) # (2)

Del bloque de codigo anterior vamos a resaltar dos puntos:
- (1) Vamos a desarrollar un Enum que nos permita ir haciendo **cambios de estado**.
- (2) Esta linea de codigo anexa en la cuenta del remitente la transaccion que se esta realizando. 

Un Enum nos ayuda a definir nuestros propios tipos de datos. En este caso, vamos a diseñar uno que nos de 3 casos de una transaccion:
- Completada
- Rechazada
- En espera

In [ ]:
from enum import Enum
class TxStatus(Enum):
    PENDIENTE = 0
    CONFIRMADA = 1
    DECLINADA = 2

El primer metodo de la clase Account es el siguiente:

In [ ]:
    def to_dict(self):
        """Exporta la transaccion en formato: dict."""
        return {
            'sender': self.sender.nickname,
            'recipient': self.recipient.nickname,
            'value': self.value,
            'time' : self.time}

Este metodo exporta un diccionario con el "encabezado" de la transaccion. Es la informacion que se codifica y pasa a ser el contenido de la transaccion.

### Firmado y Verificado de transacciones utilizando una cuenta 

#### Firmado

Digamos que nuestra transaccion es lo siguiente:

In [ ]:
transaction = {
    "sender": "Pedro",
    "receiver": "maria",
    "amount": 10
}

In [ ]:
# Si trataramos de ingresar en un hash este contenido pasaria lo siguiente:
from Crypto.Hash import SHA256
# hash = SHA256.new(transaction) # Sale un errorsaso.

# SHA256 solo acepta valores alfanumericos, por lo tanto necesitamos convertir nuestro diccionario en una cadena de bites
# Primero: dict -> str
transaction_str = str(transaction)

# Ya lo convertimos a String, ahora necesitamos convertirlo a bites
transaction_byte = transaction_str.encode()
type(transaction)

In [ ]:
# Una vez lo tenemos en bytes podemos pasarlo por un algoritmo SHA256 sin problema.
tx_hashed = SHA256.new(transaction_byte)

# Imprimimos el hash en su version hexadecimal
tx_hashed.hexdigest()

Ya que tenemos el contenido de nuestra transaccion y su hash, podemos obtener su firma.

In [ ]:
# Hagamos el ejemplo completo:
# Pedro es el dueno de la transaccion.
# Por lo tanto, Pedro tiene una cuenta con sus respectivas llaves.
from Account import Account
pedro = Account('pedro')

# Gracias a que desarrollamos la clase cuenta, podemos utilizarla en este ejemplo.
# Pedro tiene su llave privada y pubica, asi como su firmador y su verificador.
# Pedro procede a firmar la transaccion
signature = pedro.signer.sign(tx_hashed)

# Nuestra firma se ve de la siguiente manera
signature

#### Verificado

Para verificar una transaccion el proceso es muy similar. 

In [ ]:
# Necesitamos pasar por un hash el contenido de la transaccion.
tx_hashed = SHA256.new(str(transaction).encode()) # Todo el hash en una sola linea de codigo

# Ahora, en vez de usar el firmador, vamos a usar el verificador. 
try:
    pedro.verifier.verify(tx_hashed, signature)
    print(True)
except:
    print(False)


El verificador alza un error si la firma es invalida. Por eso utilizamos try/except

In [ ]:
# Si lo intentaramos con una firma falsa el resultado seria distinto
firma_falsa = SHA256.new('transaccion falsa'.encode())
# Ahora, en vez de usar el firmador, vamos a usar el verificador. 
try:
    pedro.verifier.verify(firma_falsa, signature)
    print(True)
except:
    print(False)

Las condiciones para que salga error son:
- El contenido de la transaccion haya sido alterado.
- La firma es incorrecta.
- Quien firma no es el autor de la transaccion.

Vamos a anadir estas funciones a los metodos de nuestra clase Transaction. Nuestro codigo completo quedaria asi:

In [ ]:
from datetime import datetime
from enum import Enum
from Account import Account
from Crypto.Signature.pkcs1_15 import PKCS115_SigScheme
from Crypto.Hash import SHA256

class TxStatus(Enum):
    PENDIENTE = 0
    CONFIRMADA = 1
    DECLINADA = 2

class Transaction:
    def __init__(self, sender: Account, value: int, recipient: Account):
        self.sender = sender
        self.value = value
        self.recipient = recipient
        self.time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        self.block = None
        self.signature = None
        self.status = TxStatus.PENDIENTE
        # Al instanciarse una tx, esta debe reflejarse en la cuenta que la envia. (Sender)
        sender.list_of_all_transactions.append(self)


    def to_dict(self):
        """Exporta la transaccion en formato: dict."""
        return {
            'sender': self.sender.nickname,
            'recipient': self.recipient.nickname,
            'value': self.value,
            'time' : self.time}

    def sign_transaction(self): # (1)
        """Funcion que recibe un objeto transaccion y devuelve 
        la firma de la transaccion en bytes"""
        print("Firmando transaccion...")
        msg = str(self.to_dict()).encode()
        hash = SHA256.new(msg)
        signer = self.sender.signer
        signature = signer.sign(hash)
        # print("Signature:", binascii.hexlify(signature))
        self.signature = signature

    def verify_signature(self) -> bool: # (1)
        """Aqui se verifican las transacciones"""
        print("Verificando la firma de la transaccion...")
        msg = str(self.to_dict()).encode()
        hash = SHA256.new(msg)
        verifier = self.sender.verifier
        try:
            verifier.verify(hash, self.signature)
            print("La firma es valida.")
            return True
        except:
            print("La firma es invalida.")
            return False

    def change_status(self, new_status): # (3)
        if new_status == 'CONFIRMADA':
            self.status = TxStatus.CONFIRMADA
        elif new_status == 'PENDIENTE':
            self.status = TxStatus.PENDIENTE
        elif new_status == 'DECLINADA':
            self.status = TxStatus.DECLINADA

Lo que aplicamos se encuentra en las funciones sign_transaction(), verify_signature() y to_dict(). La unica funcion que nos falto comentar es change_status()

#### Funcion change_status()

Funcion que cambia el estado de una transaccion. Hay que recordar que los estados cambian dependiendo del evento, por ejemplo:
- Cuando una transaccion se crea, empieza siendo Pendiente.
- Cuando es parte de la cadena de bloques, pasa a estar confirmada.
- Si la red tiene algun error, podria rechazar la transaccion.

## Clase Blockchain

Llego la hora de definir nuestra blockchain. Ya estan casi todas la piezas y solo queda ir ensamblando poco. Vayamos a hacerlo.